## Trajectory Calculations - workflow

- first to calculate the trajectory, the reverse trajectory, and the clipped trajectories
- then to calculate the integration error and filter out the bad trajectories
- seperate the trajectories into folders which are rainy, dry and total, then seasonally?

##### Mapping
- map the different groups

In [1]:
# STEP 0: Must be done before anything each time notebook is loaded
# Import PySplit

import pysplit as py
import numpy as np
import pickle as pickle
from pprint import pprint
import os
from __future__ import print_function
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

#### Trajectory generation

- 1980 to 2017 is generated, group created for 2000-2017 at the moment (due to size of the analysis)
- Jan to Dec
- 4th hour
- altitude of 1500 - best for Christmas Island, ends at 850hpa which is rainfall level. The endpoint altitude

###### Below the years of 2015-2017 have been calculated as a start 
- nail out the clustering and then use the base code to create a climatology based on these years, and then look at the differences between the different seasons in Christmas Island 




In [2]:
# STEP 1: TRAJECTORY GENERATION (not yet tested on mac)
# check out bulk_trajen_example.py

#directory setup
# working_dir = r'/Volumes/Seagate_Backup/hysplit4/working'
# storage_dir = r'/Volumes/Seagate_Backup/hysplit4/pysgen/XI'
# meteo_dir = r'/Volumes/Seagate_Backup/HYSPLIT/ncep_monYYYY'
# # meteo_dir = r'C:\hysplit4\gdas'

# #file basename
# basename = 'XI20'

# if not os.path.exists(storage_dir):
#     os.makedirs(storage_dir)
    
# #Arguments
# #years = [2007]
# years=range(2000,2018)
# #months = [1, 8]
# months=range(1,13)
# hours = [4]
# altitudes = [1500]
# location = (-10.45, 105.7)
# runtime = -168

# #Generate trajs
# py.generate_bulktraj(basename, working_dir, storage_dir, meteo_dir,
#                           years, months, hours, altitudes, location, runtime, meteoyr_2digits=False,
#                           monthslice=slice(0, 32, 1), meteo_bookends=([0],[0]), get_reverse=True, get_clipped=True, hysplit="/Volumes/Seagate_Backup/hysplit4/exec/hyts_std")

#### Grouping the trajectories

- determine the distance between the start and end of trajectories, the vector 
- first seperated by the names - the summer/winter etv values


In [4]:
# selecting only the XI_20 data set - this is from 2000-2017
trajgroup = py.make_trajectorygroup(r'/Volumes/Seagate_Backup/hysplit4/pygen/XI2/*20*')

for traj in trajgroup:
    traj.calculate_distance()
    traj.calculate_vector()
    traj.calculate_rh()
    traj.calculate_sh()
    
    

/Users/jess/anaconda3/envs/pysplitenv/lib/python3.6/site-packages/pysplit/hypath.py:136: RuntimeWarning: invalid value encountered in arccos
  np.cos(lon[:-1] - lon[1:])) * 6371) * 1000


In [5]:
#vars(trajgroup[3])
print('Total trajectories in group: ',trajgroup.trajcount)

Total trajectories in group:  6575


In [ ]:
# #trajgroup.make_infile(r'/Volumes/Seagate_Backup/hysplit4/cluster/working')
# vars(trajgroup)

# bad = []
# for traj in trajgroup:
#         bad.append(traj.trajid)

# print('Expectation: ', trajgroup.trajcount, 'trajectories -', len(bad),
#       'bad trajectories =', trajgroup.trajcount-len(bad), 'trajectories')


# trajgroup.pop(trajid=bad)
# print('Result: ', trajgroup.trajcount, 'trajectories')



In [ ]:
#vars(trajgroup[3])

#### Calculating the integration error

- integration error calculated for the entire period.



- first the reversetraj has to be loaded, this is because the integration error is the physical and numerical error. This is calculated by the distance between the original trajectory and the reverse trajectory start and end points. 

- then the integration error can be calculated by traj.calculate_integrationerr()

    - this also should inclue te traj.integration_error (the relative error%) and traj.integration_error_abs (the absolute error in meters)
    
    
   

In [ ]:
#Step 1: integration error filtering integr 
#first load reverse Traj
for traj in trajgroup:
    traj.load_reversetraj()
#then calculate integration error
# Values computed when calling ``Trajectory.calculate_integrationerr()``:
#     ``Trajectory.integration_error``, the relative error (%)
#     ``Trajectory.integration_error_abs``, the absolute error (meters)


In [ ]:
#Step 2: calculate the integration error and determine if there is any problems in the integration calculation 
#any problems would be to do with the reverse trajectory and any issues with the generation of this. 

c=0
for traj in trajgroup:
    traj.calculate_integrationerr()
    print(c)
    c+=1

In [ ]:
#done to identify the the bad reverse trajectories
badreverse=[] #identigy whihc of the trajectories are bad and then put them into a badreverse list of the trajectory.trajid.
# this is done through .append below

for traj in trajgroup:
    try: 
        print(traj.integration_error)
    except:
        print(traj.trajid)
        badreverse.append(traj.trajid)
        
#.pop is used to exclude a number of trajectories from the list, based on a list created previously., i.e the badreverse
#list of trajectories can be used. 

trajgroup.pop(trajid=badreverse) 
print('Result: ', trajgroup.trajcount, 'trajectories')

In [ ]:
#The cut-off is identified in order to identify trajectories which are outside of the acceptable range, 
#these can then be taken out of the relationships through the append functions below
relative_errors = [traj.integration_error for traj in trajgroup]

#cutoff = np.mean(relative_errors) + (np.std(relative_errors) * 2)
relative_errors_average=0.183888357; #calculated in excel because np.mean was producing inf and nan
relative_errors_std=1.303486797;
cutoff = relative_errors_average + (relative_errors_std * 2)

#print (np.average(relative_errors))
#print (np.std(relative_errors))
print('Integration error upper limit: ', cutoff)



In [ ]:
bad = []
for traj in trajgroup: #loop to exclude the bad trajectories in the groups - based on the integration error and cutoff of 2 standard deviations
    if traj.integration_error > cutoff:
        bad.append(traj.trajid)

print('Expectation: ', trajgroup.trajcount, 'trajectories -', len(bad),
      'bad trajectories =', trajgroup.trajcount-len(bad), 'trajectories')


# trajgroup.pop(trajid=bad)
# print('Result: ', trajgroup.trajcount, 'trajectories')



### Create the rainy, dry and total groups - also the Moisture uptakes

- also read out the infiles which will be used to conduct the clustering analysis in hypslit

In [ ]:
# # moisture uptake identification of the moisture source regions for the rainy day trajectories
# moisture = []
# for traj in trajgroup:
#     traj.moisture_uptake(precipitation=-0.2,evaporation=0.2,interval=3)
#     moisture.append(traj)
    

In [ ]:
#appending the rainy day trajectories
rainylist = []

for traj in trajgroup:
    traj.set_rainstatus()
    if traj.rainy:
        rainylist.append(traj)
        
rainy_trajgroup = py.TrajectoryGroup(rainylist)
dry_trajgroup = trajgroup - rainy_trajgroup
print('Total trajectories in total group: ',trajgroup.trajcount)
print('Total trajectories in dry group: ',dry_trajgroup.trajcount)
print('Total trajectories in rainy group: ',rainy_trajgroup.trajcount)


In [ ]:
# saving out the list of rainy trajectories 
#rainy_trajgroup.make_infile(r'/Volumes/Seagate_Backup/hysplit4/cluster/working')
#dry_trajgroup.make_infile(r'/Volumes/Seagate_Backup/hysplit4/cluster/working')
#trajgroup.make_infile(r'/Volumes/Seagate_Backup/hysplit4/cluster/working')

## Mapping examples

- https://github.com/mscross/pysplit/blob/master/docs/examples/basic_plotting_example.py
- https://github.com/mscross/pysplit/blob/master/docs/examples/basic_scatterplotting.py


- testing a number of different plots and determine what is the best representation for this analysis 

In [ ]:
import matplotlib.pyplot as plt

## Clustering of the different trajectory groups 

- taken from hysplit_clustering from mscross 
- done for the clustering developed from the rainy_day and the Dry_days

In [ ]:
traj_assignment = r'/Volumes/Seagate_Backup/hysplit4/cluster/archive/Rainy Cluster/CLUSLIST_4'
clusterpath_dir = r'/Volumes/Seagate_Backup/hysplit4/cluster/archive/Rainy Cluster'

clusgroup = py.spawn_clusters(rainy_trajgroup, traj_assignment, clusterpath_dir)

In [ ]:
colors = np.linspace(0, 0.95, 7)

mapcorners =  [50, -40, 170, 10]
standard_pm = None
mapdesign0 = py.MapDesign(mapcorners, standard_pm, projection='cyl',
                 mapcolor='light', maplabels=None, area_threshold=10000,
                 resolution='c', zborder=14,
                 zmapbound=10, zlatlon=11, lat_labels=['left'],
                 lon_labels=['top'], latlon_labelspacing=(10, 20),
                 latlon_fs=20, latlon_spacing=(10, 20), drawstates=False,
                 drawoutlines=True, draw_latlons=True, land_alpha=0.85)
mapdesign1 = py.MapDesign(mapcorners, standard_pm, lon_labels=['bottom'])

fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(20,20))
map0 = mapdesign0.make_basemap(ax=ax0)
map1 = mapdesign1.make_basemap(ax=ax1)

# Text on maps
x, y = map0(-135, 25)
font_params = {'horizontalalignment' : 'center',
               'verticalalignment' : 'center',
               'fontsize' : 40,
               'weight' : 'bold'}

ax0.text(x, y, 'Clusters', **font_params)
ax1.text(x, y, 'Trajectories', **font_params)

for clus, color in zip(clusgroup, colors):
    params = {'zorder' : 24,
    		  'latlon' : True,
    		  'c' : plt.cm.magma(color)}

    map0.plot(*clus.path.xy, lw=(clus.trajcount/clusgroup.trajcount)*50,
    		  **params)

    for traj in clus:
        map1.plot(*traj.path.xy, lw=1.5, **params)
        
fig.subplots_adjust(hspace=0.1)

In [ ]:
mapcorners =  [50, -40, 170, 10]
standard_pm = None

bmap_params = py.MapDesign(mapcorners, standard_pm, projection='cyl',
                 mapcolor='light', maplabels=None, area_threshold=10000,
                 resolution='c', zborder=14,
                lat_labels=['left'],
                 lon_labels=['top'], latlon_labelspacing=(10, 20),
                 latlon_fs=20, latlon_spacing=(10, 20), drawstates=False,
                 drawoutlines=True, draw_latlons=True)
bmap = bmap_params.make_basemap()
for traj in clusgroup[::]:
    bmap.plot(*traj.path.xy, latlon=True, zorder=20)